In [ ]:
import requests
import pandas as pd
    
pdHtml = pd.read_html("http://landtop.com.tw/products.php?types=1",encoding="utf-8")
pdHtml[0]

In [44]:
import requests
import csv

html = requests.get("http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=20181206&type=ALLBUT0999&_=1544171814819")

with open("stock.csv","w",encoding="utf-8-sig") as fp:
    fp.write(html.text)
with open("stock.csv","r",encoding="utf-8-sig") as fp:
    content = list(csv.reader(fp))

In [66]:
content[1000]

['2017',
 '官田鋼',
 '185844',
 '90',
 '1526442',
 '8.30',
 '8.30',
 '8.18',
 '8.18',
 '-',
 '0.10',
 '8.18',
 '2',
 '8.19',
 '2',
 '12.78']

In [45]:
# 找出content裡有17個欄位的元素，加到stockLi裡
stockLi = []
for i in content:
    if len(i) == 17:
        del i[-1]
        stockLi.append(i)

In [48]:
stockLi

['證券代號',
 '證券名稱',
 '成交股數',
 '成交筆數',
 '成交金額',
 '開盤價',
 '最高價',
 '最低價',
 '收盤價',
 '漲跌(+/-)',
 '漲跌價差',
 '最後揭示買價',
 '最後揭示買量',
 '最後揭示賣價',
 '最後揭示賣量',
 '本益比']

In [50]:
# 刪除第 0 列
del stockLi[0]

In [56]:
stockLi

[['證券代號',
  '證券名稱',
  '成交股數',
  '成交筆數',
  '成交金額',
  '開盤價',
  '最高價',
  '最低價',
  '收盤價',
  '漲跌(+/-)',
  '漲跌價差',
  '最後揭示買價',
  '最後揭示買量',
  '最後揭示賣價',
  '最後揭示賣量',
  '本益比'],
 ['0050',
  '元大台灣50',
  '20347421',
  '10519',
  '1529965270',
  '75.85',
  '75.85',
  '74.85',
  '75.10',
  '-',
  '1.55',
  '75.10',
  '88',
  '75.15',
  '637',
  '0.00'],
 ['0051',
  '元大中型100',
  '22180',
  '17',
  '643292',
  '29.48',
  '29.48',
  '28.89',
  '28.89',
  '-',
  '0.80',
  '28.89',
  '1',
  '29.00',
  '20',
  '0.00'],
 ['0052',
  '富邦科技',
  '107000',
  '21',
  '5134770',
  '48.42',
  '48.42',
  '47.79',
  '47.97',
  '-',
  '1.36',
  '47.89',
  '1',
  '48.20',
  '1',
  '0.00'],
 ['0053',
  '元大電子',
  '10000',
  '8',
  '307920',
  '31.21',
  '31.21',
  '30.50',
  '30.50',
  '-',
  '0.94',
  '30.40',
  '1',
  '30.57',
  '20',
  '0.00'],
 ['0054',
  '元大台商50',
  '21000',
  '16',
  '414770',
  '20.25',
  '20.25',
  '19.58',
  '19.58',
  '-',
  '0.67',
  '19.57',
  '1',
  '19.62',
  '20',
  '0.00'],
 ['0055',
  '元

In [58]:
# 整理資料裡無用的符號，字串的replace()會成為新的元素，所以要重新寫回

for i in range(len(stockLi)):
    stockLi[i][0] = stockLi[i][0].replace('"','')
    stockLi[i][0] = stockLi[i][0].replace('=','')
    for j in range(2,16):
        stockLi[i][j] = stockLi[i][j].replace(',','')

In [63]:
for i in stockLi:
    if i[5]=='--':
        print(i)

['00643K', '群益深証中小+R', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '2.31', '10', '2.37', '10', '0.00']
['00661', '元大日經225', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '26.85', '3', '27.09', '50', '0.00']
['00707R', '元大S&P日圓反1', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '20.43', '20', '20.50', '30', '0.00']
['00710B', 'FH彭博高收益債', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '20.00', '1', '20.06', '20', '0.00']
['008201', 'BP上證50', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '89.15', '1', '90.80', '1', '0.00']
['01003T', '兆豐新光R1', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '14.88', '81', '14.97', '31', '0.00']
['1418', '東華', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '6.32', '1', '7.12', '1', '0.00']
['2923', '鼎固-KY', '0', '0', '0', '--', '--', '--', '--', ' ', '0.00', '--', '', '--', '', '50.00']
['3701', '大眾控', '1161', '4', '13989', '--', '--', '--', '--', ' ', '0.00', '12.15', '6', '12.50', '2', '21.19']
['910708', '恒大健-DR', 

In [64]:
# 找出有開盤且收益大於 1.05 的股票
for i in range(1,len(stockLi)):
    if stockLi[i][5] != '--' and (float(stockLi[i][8])/float(stockLi[i][5]))>1.05:
        print(stockLi[i])

['2236', '百達-KY', '812844', '390', '26558120', '31.00', '32.85', '30.80', '32.85', '+', '2.95', '32.85', '403', '--', '0', '11.02']
['5259', '清惠', '33350', '23', '258510', '7.32', '8.00', '7.23', '8.00', '+', '0.06', '7.30', '5', '8.00', '2', '0.00']
['6558', '興能高', '9839954', '5255', '386820367', '38.00', '42.00', '38.00', '41.80', '+', '4.30', '41.60', '8', '41.80', '38', '21.88']
['9914', '美利達', '1748121', '1230', '242841940', '133.00', '141.00', '132.50', '140.50', '+', '7.00', '140.00', '4', '140.50', '6', '36.21']
